In [1]:
#Imports
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
import warnings
import scipy.stats as scs
import statistics
import math
import seaborn as sns
%matplotlib inline
import random
from collections import Counter

In [2]:
#
df=pd.read_csv(r"C:\Users\vivak\Documents\vos\coding_snps_freqs.txt", sep='\t', header=0)
df['nonAFR']=df.EAS+df.EUR+df.SAS
for i in ['AFR','EAS','EUR','SAS','nonAFR']:
    df[i] = np.where(df[i]>0.5, 1-df[i], df[i])
    
df['csq'] = np.where(df['csq'].str.contains('synonymous'), 's', 'ns')

In [5]:
#Get shared polymorphism counts
def getShared(df, p1, p2):
    shared = []
    for i in np.linspace(0,0.4,5):
        s=df[(df[p1]>i) & (df[p1]<=i+0.1) & (df[p2]>0)].csq.value_counts()
        if('s' not in s.index):
            s['s']=0
        if('ns' not in s.index):
            s['ns']=0
        shared.append(np.array([s.ns,s.s]))
    #Repeat for 0.1-0.5 maf      
    s=df[(df[p1]>0.1) & (df[p1]<=0.5) & (df[p2]>0)].csq.value_counts()
    if('s' not in p.index):
        s['s']=0
    if('ns' not in p.index):
        s['ns']=0
    shared.append(np.array([s.ns,s.s]))    
    return(shared)

In [6]:
#Get private polymorphism counts
def getPrivate(df, p1, p2):
    private = []
    for i in np.linspace(0,0.4,5):
        p=df[(df[p1]>i) & (df[p1]<=i+0.1) & (df[p2]==0)].csq.value_counts()
        if('s' not in p.index):
            p['s']=0
        if('ns' not in p.index):
            p['ns']=0
        private.append(np.array([p.ns,p.s]))
        
    p=df[(df[p1]>0.1) & (df[p2]==0)].csq.value_counts()
    if('s' not in p.index):
        p['s']=0
    if('ns' not in p.index):
        p['ns']=0
    private.append(np.array([p.ns,p.s]))
    return(private)

In [8]:
#Bootstrap results to get CIs
def bootstrapper(df):
    lp = []
    up = []
    v=[]

    for i,j in enumerate(df['rN']):
        counts=[df['rN'][i],df['rS'][i],df['sN'][i],df['sS'][i]]
        total_snps=sum(counts)
        bZ_list=[]

        for i in range(0, 1000):
            btstrp = np.random.choice(['sN', 'sS', 'rN', 'rS'], total_snps, 
                             p=[counts[2]/total_snps, counts[3]/total_snps,
                                counts[0]/total_snps, counts[1]/total_snps])

            names, bCounts = np.unique(btstrp, return_counts=True)
            if(len(bCounts)<4):
                bZ_list.append(0)
            else:
                bZ_list.append((bCounts[2]/bCounts[3]) / (bCounts[0]/bCounts[1]))

        bZ_sorted = sorted(bZ_list)
        lp.append(bZ_sorted[249])
        up.append(bZ_sorted[749])
        v.append(np.var(bZ_sorted))
    df['variance']=v
    df['CI_low']=lp
    df['CI_high']=up
    
    return(df)   

In [9]:
#Calculate Z
def getZ(df, p1, p2, outFile=""):
    res= pd.DataFrame()
    s=getShared_avg(df,p1,p2)
    p=getPrivate(df,p1,p2)    
    z=pd.merge(pd.DataFrame(s, columns=['sN','sS']),pd.DataFrame(p, columns=['rN','rS']), left_index=True, right_index=True)
    z['Z']=np.where((z.sS>0) & (z.rN>0) & (z.rS>0), (z.sN/z.sS)/(z.rN/z.rS), 0)
    z=bootstrapper(z)
    z['population']=p1
    z['maf']=['0-0.1','0.1-0.2','0.2-0.3','0.3-0.4','0.4-0.5','0.1-0.5']
    res=pd.concat([res,z])
    s=getShared_avg(df,p2,p1)
    p=getPrivate(df,p2,p1)    
    z=pd.merge(pd.DataFrame(s, columns=['sN','sS']),pd.DataFrame(p, columns=['rN','rS']), left_index=True, right_index=True)
    z['Z']=np.where((z.sS>0) & (z.rN>0) & (z.rS>0), (z.sN/z.sS)/(z.rN/z.rS), 0)
    z=bootstrapper(z)
    z['population']=p2
    z['maf']=['0-0.1','0.1-0.2','0.2-0.3','0.3-0.4','0.4-0.5','0.1-0.5']
    res=pd.concat([res,z])
    #Only if outfile parameter exits do we write out to file
    if(outFile==""):
        return(res)
    
    else:
        res.to_csv(outFile, header=True, index=True, sep='\t')
        return(res)

In [11]:
def plotZ_comparison(df, outFile=""):
    df.index=df.maf
    df.index.name='MAF'
    fig, ax = plt.subplots(figsize=(24,8))
    error=[df["Z"]-df['CI_low'], df['CI_high']-df["Z"]]
    plt.subplot(121)
    ax = sns.pointplot(df.index, 'Z', hue='population',
        data=df, dodge=True, join=False, ci=None, legend=False)
    # Find the x,y coordinates for each point
    x_coords = []
    y_coords = []
    for point_pair in ax.collections:
        for x, y in point_pair.get_offsets():
            x_coords.append(x)
            y_coords.append(y)

    # Calculate the type of error to plot as the error bars
    # Make sure the order is the same as the points were looped over
    ax.errorbar(x_coords, y_coords, yerr=error, fmt=' ', zorder=-1,
               solid_capstyle='projecting', capsize=6, ecolor='k')
    ax.xlabel="minor allele frequency"
    plt.axhline(y=1.0, color='g', linestyle='--')
    plt.rcParams.update({'font.size': 12})
    
    if(outFile!=""):
        plt.savefig(outFile)    

In [12]:
p1=['AFR', 'AFR', 'AFR', 'EUR', 'EAS', 'EUR','AFR']
p2=['EAS', 'EUR', 'SAS', 'EAS', 'SAS', 'SAS','nonAFR']
for i,j in enumerate(p1):
    res = getZ(df,j,p2[i])
    res['alpha'] = np.where(res.Z>1, 1-(1/res.Z), 0)
    res['alpha_low'] = np.where(res.CI_low>1, 1-(1/res.CI_low), 0)
    res['alpha_high'] = np.where(res.CI_high>1, 1-(1/res.CI_high), 0)
    
    for x in ['', '_low', '_high']:
        res['bps'+x] = np.where(res['alpha'+x]!=0, res['alpha'+x] * res.sN, 0)
        
    res.to_csv(r"C:\Users\vivak\Documents\vos\tables/"+j+"_"+p2[i]+"_strictMask.txt", 
               sep='\t', header=True, index=False)